<a href="https://colab.research.google.com/github/yhc-666/hate-speech-detection/blob/master/xgbt_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn import metrics


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [5]:
df_dataset = pd.read_csv('/content/drive/MyDrive/50.007kaggle/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/50.007kaggle/test.csv')
df_dataset_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/train_tfidf_features.csv")
df_test_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/test_tfidf_features.csv")
samplesubmit = pd.read_csv("/content/drive/MyDrive/50.007kaggle/sample_submission.csv")

In [6]:
print(samplesubmit.shape)

(4296, 2)


In [7]:
labels = df_dataset_features.iloc[:,[1]]
features = df_dataset_features.iloc[:,2:]
#y_test = df_test.iloc[:,[1]]
#X_test = df_test_features.iloc[:,1:]

#分训练集测试集
X_train, X_validation, y_train, y_validation = train_test_split(features, labels, test_size=0.05, random_state=0)

In [8]:
print(y_validation)
print(y_train)
print(X_train)
print(X_validation)

       label
5232       0
8264       0
14494      0
127        1
1603       1
...      ...
6643       0
6353       0
1933       0
11199      1
122        0

[1719 rows x 1 columns]
       label
8108       0
14580      0
11591      1
5831       0
4030       0
...      ...
9225       1
13123      0
9845       1
10799      0
2732       0

[15465 rows x 1 columns]
         0    1    2    3    4    5    6    7    8    9  ...  4990  4991  \
8108   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
14580  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
11591  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
5831   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4030   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
9225   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
13123  0.0  0.0  0.0  0.0  0.

In [32]:
model = XGBClassifier(learning_rate=0.01,
                      n_estimators=350,          # 树的个数-10棵树建立xgboost
                      max_depth=100,               # 树的深度
                      min_child_weight = 1,      # 叶子节点最小权重
                      gamma=0.,                  # 惩罚项中叶子结点个数前的参数
                      reg_lambda = 1.2,
                      subsample=0.6,             # 所有样本建立决策树
                      colsample_btree=1,         # 所有特征建立决策树
                      scale_pos_weight=2,        # 解决样本个数不平衡的问题
                      random_state=1,           # 随机数
                      slient = 0
                      )

In [ ]:
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
y_validation, y_pred_validation = y_validation, model.predict(X_validation)
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  
print("validation_acc: %.4g"% metrics.accuracy_score(y_validation, y_pred_validation))

0.6399999999999999
0.729495669893021
validation_macro_f1 : 0.6847
validation_acc: 0.6911


In [28]:
X_test = df_test_features.iloc[:,1:]

In [17]:
print(X_test)

        0    1    2    3    4    5    6    7    8    9  ...  4990  4991  4992  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
4291  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4292  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4293  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4294  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4295  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   

      4993  4994  4995  499

In [29]:
y_test = model.predict(X_test)

In [30]:
print(y_test)

[1 1 1 ... 1 0 0]


In [31]:
x = df_test
x = x.set_index("id")
x = x.drop("post", axis = 1)
x["label"] = y_test
print(x)
x.to_csv("/content/drive/MyDrive/50.007kaggle/submit.csv")

       label
id          
17185      1
17186      1
17187      1
17188      0
17189      0
...      ...
21476      1
21477      1
21478      1
21479      0
21480      0

[4296 rows x 1 columns]
